In [70]:
from numba import jit, njit, int8, int16, int32, int64, char

from numba import types
from numba.typed import Dict
import numba

In [134]:
import numpy as np
import cProfile
import pstats
from pstats import SortKey

## Pure Python function

In [141]:
nucleotides = "ACGT"

def kmers_dic(n, choice=nucleotides):
    return {a: 0.0 for a in combinaisons(choice, n)}

def combinaisons(combi, n, instances=nucleotides):
    if n == 1:
        return combi
    else:
        return [f"{a}{n}" for a in combinaisons(combi, n-1) for n in instances]
    
def seq_to_window(seq, window_size=4):
    """ Return a sliding window from a string """
    for i in range(len(seq) - window_size + 1):
        yield seq[i:i+window_size]
        
def seq_count_kmer(seq, kmer_count=None, k=4, ignore_N=True):
    if kmer_count is None:
        kmer_count = kmers_dic(k)
    wrong_base = "N"*k
    kmer_count[wrong_base] = 0.0
    for i in range(len(seq) - k + 1):
        try:
            kmer_count[seq[i:i+k]] += 1
        except:
            kmer_count[wrong_base] += 1

    if ignore_N:
        kmer_count.pop(wrong_base)
    return kmer_count

In [142]:
seq_count_kmer(long_str)

{'AAAA': 155.0,
 'AAAC': 97.0,
 'AAAG': 88.0,
 'AAAT': 114.0,
 'AACA': 55.0,
 'AACC': 66.0,
 'AACG': 76.0,
 'AACT': 69.0,
 'AAGA': 92.0,
 'AAGC': 63.0,
 'AAGG': 45.0,
 'AAGT': 45.0,
 'AATA': 55.0,
 'AATC': 79.0,
 'AATG': 69.0,
 'AATT': 62.0,
 'ACAA': 52.0,
 'ACAC': 26.0,
 'ACAG': 42.0,
 'ACAT': 38.0,
 'ACCA': 75.0,
 'ACCC': 46.0,
 'ACCG': 80.0,
 'ACCT': 38.0,
 'ACGA': 39.0,
 'ACGC': 73.0,
 'ACGG': 49.0,
 'ACGT': 50.0,
 'ACTA': 17.0,
 'ACTC': 32.0,
 'ACTG': 83.0,
 'ACTT': 26.0,
 'AGAA': 76.0,
 'AGAC': 44.0,
 'AGAG': 39.0,
 'AGAT': 53.0,
 'AGCA': 64.0,
 'AGCC': 49.0,
 'AGCG': 70.0,
 'AGCT': 43.0,
 'AGGA': 39.0,
 'AGGC': 68.0,
 'AGGG': 32.0,
 'AGGT': 43.0,
 'AGTA': 30.0,
 'AGTC': 24.0,
 'AGTG': 48.0,
 'AGTT': 59.0,
 'ATAA': 55.0,
 'ATAC': 34.0,
 'ATAG': 15.0,
 'ATAT': 46.0,
 'ATCA': 89.0,
 'ATCC': 53.0,
 'ATCG': 79.0,
 'ATCT': 59.0,
 'ATGA': 88.0,
 'ATGC': 70.0,
 'ATGG': 61.0,
 'ATGT': 26.0,
 'ATTA': 58.0,
 'ATTC': 51.0,
 'ATTG': 62.0,
 'ATTT': 56.0,
 'CAAA': 82.0,
 'CAAC': 65.0,
 'CAAG':

In [144]:
%%timeit
seq_count_kmer(long_str)

5.74 ms ± 51.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Profiling

In [163]:
%prun seq_count_kmer(long_str)

In [164]:
%snakeviz seq_count_kmer(long_str)

 
*** Profile stats marshalled to file 'C:\\Users\\Earwelen\\AppData\\Local\\Temp\\tmpkv7br82i'. 
Embedding SnakeViz in this document...


In [152]:
path_profile_original = "stats-original"
cProfile.run("seq_count_kmer(long_str*100)", path_profile_original)

In [153]:
p = pstats.Stats(path_profile_original)

In [155]:
p.sort_stats(SortKey.CUMULATIVE).print_stats(10)

Wed May 13 16:33:44 2020    stats-original

         15 function calls (12 primitive calls) in 0.586 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.586    0.586 {built-in method builtins.exec}
        1    0.001    0.001    0.586    0.586 <string>:1(<module>)
        1    0.585    0.585    0.585    0.585 <ipython-input-141-a7ecb29fedd9>:17(seq_count_kmer)
        1    0.000    0.000    0.000    0.000 <ipython-input-141-a7ecb29fedd9>:3(kmers_dic)
      4/1    0.000    0.000    0.000    0.000 <ipython-input-141-a7ecb29fedd9>:6(combinaisons)
        3    0.000    0.000    0.000    0.000 <ipython-input-141-a7ecb29fedd9>:10(<listcomp>)
        1    0.000    0.000    0.000    0.000 <ipython-input-141-a7ecb29fedd9>:4(<dictcomp>)
        1    0.000    0.000    0.000    0.000 {method 'pop' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    

In [156]:
p.sort_stats(SortKey.TIME).print_stats(20)

Wed May 13 16:33:44 2020    stats-original

         15 function calls (12 primitive calls) in 0.586 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.585    0.585    0.585    0.585 <ipython-input-141-a7ecb29fedd9>:17(seq_count_kmer)
        1    0.001    0.001    0.586    0.586 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 <ipython-input-141-a7ecb29fedd9>:10(<listcomp>)
        1    0.000    0.000    0.000    0.000 <ipython-input-141-a7ecb29fedd9>:4(<dictcomp>)
        1    0.000    0.000    0.586    0.586 {built-in method builtins.exec}
      4/1    0.000    0.000    0.000    0.000 <ipython-input-141-a7ecb29fedd9>:6(combinaisons)
        1    0.000    0.000    0.000    0.000 <ipython-input-141-a7ecb29fedd9>:3(kmers_dic)
        1    0.000    0.000    0.000    0.000 {method 'pop' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.

In [157]:
%load_ext snakeviz

## Numba adaptation

In [71]:
numba.NUMBA_DEBUGINFO = 1

In [13]:
k_value = 4

In [11]:
dict_param = Dict.empty(
    key_type=types.unicode_type,
    value_type=types.float64,
)

In [12]:
def combinaisons(combi, n, instances=nucleotides):
    if n == 1:
        return combi
    else:
        return [a+n for a in combinaisons(combi, n-1) for n in instances]

In [103]:
# @njit
def seq_to_window(seq, window_size=4):
    """ Return a sliding window from a string """
    for i in range(len(seq) - window_size + 1):
        yield seq[i:i+window_size]

In [109]:
def init_kmers_dic(k, choice=nucleotides):
    dic = Dict.empty(
        key_type=types.unicode_type,
        value_type=types.float64,
    )
    dic["N"*k] = 0.0
    for kmer in combinaisons(nucleotides, k):
        dic[kmer] = 0.0
    return dic

In [110]:
kmers_dic = init_kmers_dic(k_value)

In [94]:
def kmer_counter(seq, kmers_dic, k=k_value):
    for i in range(len(seq) -k +1):
        kmers_dic[seq[i:i+k]] += 1
    return kmers_dic

In [113]:
def kmer_counter(seq, kmers_dic, k=k_value, ignore_N=True):
    wrong_base = "N"*k # "".join(["N"] * k)
#     print(wrong_base)
    for i in range(len(seq) -k +1):
        try:
            kmers_dic[seq[i:i+k]] += 1
        except:
            kmers_dic[wrong_base] += 1

    if ignore_N:
        kmers_dic.pop(wrong_base)
#     return  kmers_dic

In [121]:
def kmer_counter(seq, kmers_dic, k=k_value, ignore_N=False):
    wrong_base = "N"*k # "".join(["N"] * k)
#     print(wrong_base)
    for sub_string in seq_to_window(seq, k):
        try:
            kmers_dic[sub_string] += 1
        except:
            kmers_dic[wrong_base] += 1

    if ignore_N:
        kmers_dic.pop(wrong_base)
#     return kmers_dic

In [115]:
nb_kmer_counter = njit()(kmer_counter)

In [116]:
kmer_counter(long_str, init_kmers_dic(k_value), k_value)

In [117]:
nb_kmer_counter(long_str, init_kmers_dic(k_value), k_value)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'seq_to_window':[0m [1m[1mcannot determine Numba type of <class 'function'>[0m
[1m
File "<ipython-input-114-52e4d1ac12a8>", line 4:[0m
[1mdef kmer_counter(seq, kmers_dic, k=k_value, ignore_N=True):
    <source elided>
#     print(wrong_base)
[1m    for sub_string in seq_to_window(seq, k):
[0m    [1m^[0m[0m
[0m

In [99]:
%%timeit
kmer_counter(long_str, init_kmers_dic(k_value), k_value)

136 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [100]:
%%timeit
nb_kmer_counter(long_str, init_kmers_dic(k_value), k_value)

6.66 ms ± 222 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [108]:
%%timeit
kmer_counter(long_str, init_kmers_dic(k_value), k_value)

139 ms ± 3.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [100]:
%%timeit
nb_kmer_counter(long_str, init_kmers_dic(k_value), k_value)

6.66 ms ± 222 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [354]:
k_value = 4

In [123]:
kmers_dic = init_kmers_dic(k_value)

In [124]:
%%timeit
kmer_counter(long_str, kmers_dic, k_value)

136 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [125]:
kmers_dic = init_kmers_dic(k_value)

In [126]:
%%timeit
nb_kmer_counter(long_str, kmers_dic, k_value)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'seq_to_window':[0m [1m[1mcannot determine Numba type of <class 'function'>[0m
[1m
File "<ipython-input-114-52e4d1ac12a8>", line 4:[0m
[1mdef kmer_counter(seq, kmers_dic, k=k_value, ignore_N=True):
    <source elided>
#     print(wrong_base)
[1m    for sub_string in seq_to_window(seq, k):
[0m    [1m^[0m[0m
[0m

In [254]:
code_counter(long_bytes_arr)

1232

In [255]:
nb_code_counter(longlong_bytes_arr_str)

1232

In [258]:
%%timeit
code_counter(long_bytes_arr)

3.68 ms ± 513 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [259]:
%%timeit
nb_code_counter(long_bytes_arr)

539 µs ± 8.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [390]:
nucleotides = "ACGT"

def combinaisons(combi, n, instances=nucleotides):
    if n == 1:
        return combi
    else:
        return [a+n for a in combinaisons(combi, n-1) for n in instances]

combis = combinaisons(nucleotides, 3)

def kmers_dic(n, choice=nucleotides):
    result_dict = Dict.empty(
        key_type=types.unicode_type,
        value_type=types.float64,
    )
    for a in combis:
        result_dict[a] = 0.0
    return result_dict
    
@njit
def seq_to_window(seq, window_size=4):
    """ Return a sliding window from a string """
    for i in range(len(seq) - window_size + 1):
        yield seq[i:i+window_size]

In [417]:
dic_3 = kmers_dic(k_value)

In [421]:
def seq_count_kmer(seq="", kmer_count=dic_3, k=3, ignore_N=True):
    print("non sense")
    wrong_base = "NNN"  # .join(["N"] * k)
    print("don't get it")
    kmer_count[wrong_base] = 0.0
    for kmer in seq_to_window(str(seq), k):
        try:
            kmer_count[kmer] += 1
        except:
            kmer_count[wrong_base] += 1

    if ignore_N:
        kmer_count.pop(wrong_base)
    return kmer_count

In [419]:
nb_seq_count_kmer = jit(nopython=True)(seq_count_kmer)

In [420]:
nb_seq_count_kmer(long_str)

LoweringError: Failed in nopython mode pipeline (step: nopython mode backend)
[1m[1mCannot lower constant of type 'DictType[unicode_type,float32]'
[1m
File "<ipython-input-418-9630c60edab9>", line 2:[0m
[1mdef seq_count_kmer(seq="", kmer_count=dic_3, k=3, ignore_N=True):
[1m    print("non sense")
[0m    [1m^[0m[0m
[0m
[0m[1m[1] During: lowering "kmer_count = arg(1, name=kmer_count)" at <ipython-input-418-9630c60edab9> (2)[0m

In [ ]:
%%timeit
nb_seq_count_kmer(long_str)

## Using bytes

In [64]:
def count_kmer(seq=[65, 67, 71, 84, 32, 32, 97, 99, 103, 116], k=4):
    """ numba implementation """
    match = [65, 67, 71]
    count = 0
    for i in range(len(seq) - len(match) + 1):
        if seq[i] == match[0] and seq[i+1] == match[1] and seq[i+2] == match[2]:
            count += 1
    return count

In [64]:
def count_kmer(seq=[65, 67, 71, 84, 32, 32, 97, 99, 103, 116], k=4):
    """ numba implementation """
    match = [65, 67, 71]
    count = 0
    for i in range(len(seq) - len(match) + 1):
        if seq[i] == match[0] and seq[i+1] == match[1] and seq[i+2] == match[2]:
            count += 1
    return count

In [65]:
count_kmer()

1

In [183]:
long_bytes_arr = np.frombuffer(long_str.encode(), dtype='uint8')
long_bytes_arr

array([65, 71, 67, ..., 65, 65, 65], dtype=uint8)

In [245]:
letter_code = list(bytes(b'ACGT'))
letter_code

[65, 67, 71, 84]

In [247]:
codes = np.zeros(len(letter_code)**k_value)
codes

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
for k in range(k_value):
    for code in letter_code:
        codes

In [77]:
bytearray(long_str, 'utf-8')

bytearray(b'AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCCATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCCCGCACCTGACAGTGCGGGCTTTTTTTTTCGACCAAAGGTAACGAGGTAACAACCATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTGGCGATGATTGAAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTTGACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAAAACATGTCCTGCATGGCATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGCTGATTTGCCGTGGCGAGAAAATGTCGATCGCCATTATGGCCGGCGTATTAGAAGCGCGCGGTCACAACGTTACTGTTATCGATCCGGTCGAAAAACTGCTGGCAGTGGGGCATTACCTCGAATCTACCGTCGATATTGCTGAGTCCACCCGCCGTATTGCGGCAAGCCGCATTCCGGCTGATCACATGGTGCTGATGGCAGGTTTCACCGCCGGTAATGAAAAAGGCGAACTGGTGGTGCTTGGACGCAACGGTTCCGACTACTCTGCTGCGGTGCTGGCTGCCTGTT

In [4]:
long_str = """AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGC
TTCTGAACTGGTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAA
TATAGGCATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCCATGAAACGCATTAGCACCACC
ATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAG
CCCGCACCTGACAGTGCGGGCTTTTTTTTTCGACCAAAGGTAACGAGGTAACAACCATGCGAGTGTTGAA
GTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCC
AGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTGGCGATGATTG
AAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTT
GACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTT
GCCCAAATAAAACATGTCCTGCATGGCATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGC
TGATTTGCCGTGGCGAGAAAATGTCGATCGCCATTATGGCCGGCGTATTAGAAGCGCGCGGTCACAACGT
TACTGTTATCGATCCGGTCGAAAAACTGCTGGCAGTGGGGCATTACCTCGAATCTACCGTCGATATTGCT
GAGTCCACCCGCCGTATTGCGGCAAGCCGCATTCCGGCTGATCACATGGTGCTGATGGCAGGTTTCACCG
CCGGTAATGAAAAAGGCGAACTGGTGGTGCTTGGACGCAACGGTTCCGACTACTCTGCTGCGGTGCTGGC
TGCCTGTTTACGCGCCGATTGTTGCGAGATTTGGACGGACGTTGACGGGGTCTATACCTGCGACCCGCGT
CAGGTGCCCGATGCGAGGTTGTTGAAGTCGATGTCCTACCAGGAAGCGATGGAGCTTTCCTACTTCGGCG
CTAAAGTTCTTCACCCCCGCACCATTACCCCCATCGCCCAGTTCCAGATCCCTTGCCTGATTAAAAATAC
CGGAAATCCTCAAGCACCAGGTACGCTCATTGGTGCCAGCCGTGATGAAGACGAATTACCGGTCAAGGGC
ATTTCCAATCTGAATAACATGGCAATGTTCAGCGTTTCTGGTCCGGGGATGAAAGGGATGGTCGGCATGG
CGGCGCGCGTCTTTGCAGCGATGTCACGCGCCCGTATTTCCGTGGTGCTGATTACGCAATCATCTTCCGA
ATACAGCATCAGTTTCTGCGTTCCACAAAGCGACTGTGTGCGAGCTGAACGGGCAATGCAGGAAGAGTTC
TACCTGGAACTGAAAGAAGGCTTACTGGAGCCGCTGGCAGTGACGGAACGGCTGGCCATTATCTCGGTGG
TAGGTGATGGTATGCGCACCTTGCGTGGGATCTCGGCGAAATTCTTTGCCGCACTGGCCCGCGCCAATAT
CAACATTGTCGCCATTGCTCAGGGATCTTCTGAACGCTCAATCTCTGTCGTGGTAAATAACGATGATGCG
ACCACTGGCGTGCGCGTTACTCATCAGATGCTGTTCAATACCGATCAGGTTATCGAAGTGTTTGTGATTG
GCGTCGGTGGCGTTGGCGGTGCGCTGCTGGAGCAACTGAAGCGTCAGCAAAGCTGGCTGAAGAATAAACA
TATCGACTTACGTGTCTGCGGTGTTGCCAACTCGAAGGCTCTGCTCACCAATGTACATGGCCTTAATCTG
GAAAACTGGCAGGAAGAACTGGCGCAAGCCAAAGAGCCGTTTAATCTCGGGCGCTTAATTCGCCTCGTGA
AAGAATATCATCTGCTGAACCCGGTCATTGTTGACTGCACTTCCAGCCAGGCAGTGGCGGATCAATATGC
CGACTTCCTGCGCGAAGGTTTCCACGTTGTCACGCCGAACAAAAAGGCCAACACCTCGTCGATGGATTAC
TACCATCAGTTGCGTTATGCGGCGGAAAAATCGCGGCGTAAATTCCTCTATGACACCAACGTTGGGGCTG
GATTACCGGTTATTGAGAACCTGCAAAATCTGCTCAATGCAGGTGATGAATTGATGAAGTTCTCCGGCAT
TCTTTCTGGTTCGCTTTCTTATATCTTCGGCAAGTTAGACGAAGGCATGAGTTTCTCCGAGGCGACCACG
CTGGCGCGGGAAATGGGTTATACCGAACCGGACCCGCGAGATGATCTTTCTGGTATGGATGTGGCGCGTA
AACTATTGATTCTCGCTCGTGAAACGGGACGTGAACTGGAGCTGGCGGATATTGAAATTGAACCTGTGCT
GCCCGCAGAGTTTAACGCCGAGGGTGATGTTGCCGCTTTTATGGCGAATCTGTCACAACTCGACGATCTC
TTTGCCGCGCGCGTGGCGAAGGCCCGTGATGAAGGAAAAGTTTTGCGCTATGTTGGCAATATTGATGAAG
ATGGCGTCTGCCGCGTGAAGATTGCCGAAGTGGATGGTAATGATCCGCTGTTCAAAGTGAAAAATGGCGA
AAACGCCCTGGCCTTCTATAGCCACTATTATCAGCCGCTGCCGTTGGTACTGCGCGGATATGGTGCGGGC
AATGACGTTACAGCTGCCGGTGTCTTTGCTGATCTGCTACGTACCCTCTCATGGAAGTTAGGAGTCTGAC
ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCGGGTTTGATGTGCTCGGGGCGGCGG
TGACACCTGTTGATGGTGCATTGCTCGGAGATGTAGTCACGGTTGAGGCGGCAGAGACATTCAGTCTCAA
CAACCTCGGACGCTTTGCCGATAAGCTGCCGTCAGAACCACGGGAAAATATCGTTTATCAGTGCTGGGAG
CGTTTTTGCCAGGAACTGGGTAAGCAAATTCCAGTGGCGATGACCCTGGAAAAGAATATGCCGATCGGTT
CGGGCTTAGGCTCCAGTGCCTGTTCGGTGGTCGCGGCGCTGATGGCGATGAATGAACACTGCGGCAAGCC
GCTTAATGACACTCGTTTGCTGGCTTTGATGGGCGAGCTGGAAGGCCGTATCTCCGGCAGCATTCATTAC
GACAACGTGGCACCGTGTTTTCTCGGTGGTATGCAGTTGATGATCGAAGAAAACGACATCATCAGCCAGC
AAGTGCCAGGGTTTGATGAGTGGCTGTGGGTGCTGGCGTATCCGGGGATTAAAGTCTCGACGGCAGAAGC
CAGGGCTATTTTACCGGCGCAGTATCGCCGCCAGGATTGCATTGCGCACGGGCGACATCTGGCAGGCTTC
ATTCACGCCTGCTATTCCCGTCAGCCTGAGCTTGCCGCGAAGCTGATGAAAGATGTTATCGCTGAACCCT
ACCGTGAACGGTTACTGCCAGGCTTCCGGCAGGCGCGGCAGGCGGTCGCGGAAATCGGCGCGGTAGCGAG
CGGTATCTCCGGCTCCGGCCCGACCTTGTTCGCTCTGTGTGACAAGCCGGAAACCGCCCAGCGCGTTGCC
GACTGGTTGGGTAAGAACTACCTGCAAAATCAGGAAGGTTTTGTTCATATTTGCCGGCTGGATACGGCGG
GCGCACGAGTACTGGAAAACTAAATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTGC
GCAAGCCGTAACCCAGGGGTTGGGCAAAAATCAGGGGCTGTTTTTTCCGCACGACCTGCCGGAATTCAGC
CTGACTGAAATTGATGAGATGCTGAAGCTGGATTTTGTCACCCGCAGTGCGAAGATCCTCTCGGCGTTTA
TTGGTGATGAAATCCCACAGGAAATCCTGGAAGAGCGCGTGCGCGCGGCGTTTGCCTTCCCGGCTCCGGT
CGCCAATGTTGAAAGCGATGTCGGTTGTCTGGAATTGTTCCACGGGCCAACGCTGGCATTTAAAGATTTC
GGCGGTCGCTTTATGGCACAAATGCTGACCCATATTGCGGGTGATAAGCCAGTGACCATTCTGACCGCGA
CCTCCGGTGATACCGGAGCGGCAGTGGCTCATGCTTTCTACGGTTTACCGAATGTGAAAGTGGTTATCCT
CTATCCACGAGGCAAAATCAGTCCACTGCAAGAAAAACTGTTCTGTACATTGGGCGGCAATATCGAAACT
GTTGCCATCGACGGCGATTTCGATGCCTGTCAGGCGCTGGTGAAGCAGGCGTTTGATGATGAAGAACTGA
AAGTGGCGCTAGGGTTAAACTCGGCTAACTCGATTAACATCAGCCGTTTGCTGGCGCAGATTTGCTACTA
CTTTGAAGCTGTTGCGCAGCTGCCGCAGGAGACGCGCAACCAGCTGGTTGTCTCGGTGCCAAGCGGAAAC
TTCGGCGATTTGACGGCGGGTCTGCTGGCGAAGTCACTCGGTCTGCCGGTGAAACGTTTTATTGCTGCGA
CCAACGTGAACGATACCGTGCCACGTTTCCTGCACGACGGTCAGTGGTCACCCAAAGCGACTCAGGCGAC
GTTATCCAACGCGATGGACGTGAGTCAGCCGAACAACTGGCCGCGTGTGGAAGAGTTGTTCCGCCGCAAA
ATCTGGCAACTGAAAGAGCTGGGTTATGCAGCCGTGGATGATGAAACCACGCAACAGACAATGCGTGAGT
TAAAAGAACTGGGCTACACTTCGGAGCCGCACGCTGCCGTAGCTTATCGTGCGCTGCGTGATCAGTTGAA
TCCAGGCGAATATGGCTTGTTCCTCGGCACCGCGCATCCGGCGAAATTTAAAGAGAGCGTGGAAGCGATT
CTCGGTGAAACGTTGGATCTGCCAAAAGAGCTGGCAGAACGTGCTGATTTACCCTTGCTTTCACATAATC
TGCCCGCCGATTTTGCTGCGTTGCGTAAATTGATGATGAATCATCAGTAAAATCTATTCATTATCTCAAT
CAGGCCGGGTTTGCTTTTATGCAGCCCGGCTTTTTTATGAAGAAATTATGGAGAAAAATGACAGGGAAAA
AGGAGAAATTCTCAATAAATGCGGTAACTTAGAGATTAGGATTGCGGAGAATAACAACCGCCGTTCTCAT
CGAGTAATCTCCGGATATCGACCCATAACGGGCAATGATAAAAGGAGTAACCTGTGAAAAAGATGCAATC
TATCGTACTCGCACTTTCCCTGGTTCTGGTCGCTCCCATGGCAGCACAGGCTGCGGAAATTACGTTAGTC
CCGTCAGTAAAATTACAGATAGGCGATCGTGATAATCGTGGCTATTACTGGGATGGAGGTCACTGGCGCG
ACCACGGCTGGTGGAAACAACATTATGAATGGCGAGGCAATCGCTGGCACCTACACGGACCGCCGCCACC
GCCGCGCCACCATAAGAAAGCTCCTCATGATCATCACGGCGGTCATGGTCCAGGCAAACATCACCGCTAA
ATGACAAATGCCGGGTAACAATCCGGCATTCAGCGCCTGATGCGACGCTGGCGCGTCTTATCAGGCCTAC
GTTAATTCTGCAATATATTGAATCTGCATGCTTTTGTAGGCAGGATAAGGCGTTCACGCCGCATCCGGCA
TTGACTGCAAACTTAACGCTGCTCGTAGCGTTTAAACACCAGTTCGCCATTGCTGGAGGAATCTTCATCA
AAGAAGTAACCTTCGCTATTAAAACCAGTCAGTTGCTCTGGTTTGGTCAGCCGATTTTCAATAATGAAAC
GACTCATCAGACCGCGTGCTTTCTTAGCGTAGAAGCTGATGATCTTAAATTTGCCGTTCTTCTCATCGAG
GAACACCGGCTTGATAATCTCGGCATTCAATTTCTTCGGCTTCACCGATTTAAAATACTCATCTGACGCC
AGATTAATCACCACATTATCGCCTTGTGCTGCGAGCGCCTCGTTCAGCTTGTTGGTGATGATATCTCCCC
AGAATTGATACAGATCTTTCCCTCGGGCATTCTCAAGACGGATCCCCATTTCCAGACGATAAGGCTGCAT
TAAATCGAGCGGGCGGAGTACGCCATACAAGCCGGAAAGCATTCGCAAATGCTGTTGGGCAAAATCGAAA
TCGTCTTCGCTGAAGGTTTCGGCCTGCAAGCCGGTGTAGACATCACCTTTAAACGCCAGAATCGCCTGGC
GGGCATTCGCCGGCGTGAAATCTGGCTGCCAGTCATGAAAGCGAGCGGCGTTGATACCCGCCAGTTTGTC
GCTGATGCGCATCAGCGTGCTAATCTGCGGAGGCGTCAGTTTCCGCGCCTCATGGATCAACTGCTGGGAA
TTGTCTAACAGCTCCGGCAGCGTATAGCGCGTGGTGGTCAACGGGCTTTGGTAATCAAGCGTTTTCGCAG
GTGAAATAAGAATCAGCATATCCAGTCCTTGCAGGAAATTTATGCCGACTTTAGCAAAAAATGAGAATGA
GTTGATCGATAGTTGTGATTACTCCTGCGAAACATCATCCCACGCGTCCGGAGAAAGCTGGCGACCGATA
TCCGGATAACGCAATGGATCAAACACCGGGCGCACGCCGAGTTTACGCTGGCGTAGATAATCACTGGCAA
TGGTATGAACCACAGGCGAGAGCAGTAAAATGGCGGTCAAATTGGTAATAGCCATGCAGGCCATTATGAT
ATCTGCCAGTTGCCACATCAGCGGAAGGCTTAGCAAGGTGCCGCCGATGACCGTTGCGAAGGTGCAGATC
CGCAAACACCAGATCGCTTTAGGGTTGTTCAGGCGTAAAAAGAAGAGATTGTTTTCGGCATAAATGTAGT
TGGCAACGATGGAGCTGAAGGCAAACAGAATAACCACAAGGGTAACAAACTCAGCACCCCAGGAACCCAT
TAGCACCCGCATCGCCTTCTGGATAAGCTGAATACCTTCCAGCGGCATGTAGGTTGTGCCGTTACCCGCC
AGTAATATCAGCATGGCGCTTGCCGTACAGATGACCAGGGTGTCGATAAAAATGCCAATCATCTGGACAA
TCCCTTGCGCTGCCGGATGCGGAGGCCAGGACGCCGCTGCCGCTGCCGCGTTTGGCGTCGAACCCATTCC
CGCCTCATTGGAAAACATACTGCGCTGAAAACCGTTAGTAATCGCCTGGCTTAAGGTATATCCCGCCGCG
CCGCCTGCCGCTTCCTGCCAGCCAAAAGCACTCTCAAAAATAGACCAAATGACGTGGGGAAGTTGCCCGA
TATTCATTACGCAAATTACCAGGCTGGTCAGTACCCAGATTATCGCCATCAACGGGACAAAGCCCTGCAT
GAGCCGGGCGACGCCATGAAGACCGCGAGTGATTGCCAGCAGAGTAAAGACAGCGAGAATAATGCCTGTC
ACCAGCGGGGGAAAATCAAAAGAAAAACTCAGGGCGCGGGCAACGGCGTTCGCTTGAACTCCGCTGAAAA
TTATGCCATAGGCGATGAGCAAAAAGACGGCGAACAGAACGCCCATCCAGCGCATCCCCAGCCCGCGCGC
CATATACCATGCCGGTCCGCCACGAAACTGCCCATTGACGTCACGTTCTTTATAAAGTTGTGCCAGAGAA
CATTCGGCAAACGAGGTCGCCATGCCGATAAACGCGGCAACCCACATCCAAAAGACGGCTCCAGGTCCAC
CGGCGGTAATAGCCAGCGCAACGCCGGCCAGGTTGCCGCTACCCACGCGCGCCGCAAGACTGGTACACAA
TGACTGAAATGAGGTTAAACCGCCTGGCTGTGGATGAATGCTATTTTTAAGACTTTTGCCAAACTGGCGG
ATGTAGCGAAACTGCACAAATCCGGTGCGAAAAGTGAACCAACAACCTGCGCCGAAGAGCAGGTAAATCA
TTACCGATCCCCAAAGGACGCTGTTAATGAAGGAGAAAAAATCTGGCATGCATATCCCTCTTATTGCCGG
TCGCGATGACTTTCCTGTGTAAACGTTACCAATTGTTTAAGAAGTATATACGCTACGAGGTACTTGATAA
CTTCTGCGTAGCATACATGAGGTTTTGTATAAAAATGGCGGGCGATATCAACGCAGTGTCAGAAATCCGA
AACAGTCTCGCCTGGCGATAACCGTCTTGTCGGCGGTTGCGCTGACGTTGCGTCGTGATATCATCAGGGC
AGACCGGTTACATCCCCCTAACAAGCTGTTTAAAGAGAAATACTATCATGACGGACAAATTGACCTCCCT
TCGTCAGTACACCACCGTAGTGGCCGACACTGGGGACATCGCGGCAATGAAGCTGTATCAACCGCAGGAT
GCCACAACCAACCCTTCTCTCATTCTTAACGCAGCGCAGATTCCGGAATACCGTAAGTTGATTGATGATG
CTGTCGCCTGGGCGAAACAGCAGAGCAACGATCGCGCGCAGCAGATCGTGGACGCGACCGACAAACTGGC
AGTAAATATTGGTCTGGAAATCCTGAAACTGGTTCCGGGCCGTATCTCAACTGAAGTTGATGCGCGTCTT
TCCTATGACACCGAAGCGTCAATTGCGAAAGCAAAACGCCTGATCAAACTCTACAACGATGCTGGTATTA
GCAACGATCGTATTCTGATCAAACTGGCTTCTACCTGGCAGGGTATCCGTGCTGCAGAACAGCTGGAAAA
AGAAGGCATCAACTGTAACCTGACCCTGCTGTTCTCCTTCGCTCAGGCTCGTGCTTGTGCGGAAGCGGGC
GTGTTCCTGATCTCGCCGTTTGTTGGCCGTATTCTTGACTGGTACAAAGCGAATACCGATAAGAAAGAGT
ACGCTCCGGCAGAAGATCCGGGCGTGGTTTCTGTATCTGAAATCTACCAGTACTACAAAGAGCACGGTTA
TGAAACCGTGGTTATGGGCGCAAGCTTCCGTAACATCGGCGAAATTCTGGAACTGGCAGGCTGCGACCGT
CTGACCATCGCACCGGCACTGCTGAAAGAGCTGGCGGAGAGCGAAGGGGCTATCGAACGTAAACTGTCTT
ACACCGGCGAAGTGAAAGCGCGTCCGGCGCGTATCACTGAGTCCGAGTTCCTGTGGCAGCACAACCAGGA
TCCAATGGCAGTAGATAAACTGGCGGAAGGTATCCGTAAGTTTGCTATTGACCAGGAAAAACTGGAAAAA
ATGATCGGCGATCTGCTGTAATCATTCTTAGCGTGACCGGGAAGTCGGTCACGCTACCTCTTCTGAAGCC
TGTCTGTCACTCCCTTCGCAGTGTATCATTCTGTTTAACGAGACTGTTTAAACGGAAAAATCTTGATGAA
TACTTTACGTATTGGCTTAGTTTCCATCTCTGATCGCGCATCCAGCGGCGTTTATCAGGATAAAGGCATC
CCTGCGCTGGAAGAATGGCTGACATCGGCGCTAACCACGCCGTTTGAACTGGAAACCCGCTTAATCCCCG
ATGAGCAGGCGATCATCGAGCAAACGTTGTGTGAGCTGGTGGATGAAATGAGTTGCCATCTGGTGCTCAC
CACGGGCGGAACTGGCCCGGCGCGTCGTGACGTAACGCCCGATGCGACGCTGGCAGTAGCGGACCGCGAG
ATGCCTGGCTTTGGTGAACAGATGCGCCAGATCAGCCTGCATTTTGTACCAACTGCGATCCTTTCGCGTC
AGGTGGGCGTGATTCGCAAACAGGCGCTGATCCTTAACTTACCCGGTCAGCCGAAGTCTATTAAAGAGAC
GCTGGAAGGTGTGAAGGACGCTGAGGGTAACGTTGTGGTACACGGTATTTTTGCCAGCGTACCGTACTGC
ATTCAGTTGCTGGAAGGGCCATACGTTGAAACGGCACCGGAAGTGGTTGCAGCATTCAGACCGAAGAGTG
CAAGACGCGACGTTAGCGAATAAAAAAATCCCCCCGAGCGGGGGGATCTCAAAACAATTAGTGGGATTCA
CCAATCGGCAGAACGGTGCGACCAAACTGCTCGTTCAGTACTTCACCCATCGCCAGATAGATTGCGCTGG
CACCGCAGATCAGCCCAATCCAGCCGGCAAAGTGGATGATTGCGGCGTTACCGGCAATGTTACCGATCGC
CAGCAGGGCAAACAGCACGGTCAGGCTAAAGAAAACGAATTGCAGAACGCGTGCGCCTTTCAGCGTGCCG
AAGAACATAAACAGCGTAAATACGCCCCACAGACCCAGGTAGACACCAAGGAACTGTGCATTTGGCGCAT
CGGTCAGACCCAGTTTCGGCATCAGCAGAATCGCAACCAGCGTCAGCCAGAAAGAACCGTAAGAGGTGAA
TGCGGTTAAACCGAAAGTGTTGCCTTTTTTGTACTCCAGCAGACCAGCAAAAATTTGCGCGATGCCGCCG
TAGAAAATGCCCATGGCAAGAATAATACCGTCCAGAGCGAAATAACCCACGTTGTGCAGGTTAAGCAGAA
TGGTGGTCATGCCGAAGCCCATCAGGCCCAGCGGTGCCGGATTAGCCAACTTAGTGTTGCCCATAATTCC
TCAAAAATCATCATCGAATGAATGGTGAAATAATTTCCCTGAATAACTGTAGTGTTTTCAGGGCGCGGCA
TAATAATCAGCCAGTGGGGCAGTGTCTACGATCTTTTGAGGGGAAAATGAAAATTTTCCCCGGTTTCCGG
TATCAGACCTGAGTGGCGCTAACCATCCGGCGCAGGCAGGCGATTTGCAGTACGGCTGGAATCGTCACGC
GATAGGCGCTGCCGCTGACCGCTTTAACCCCATTTAGTGCCGCACCTACAGGGCCTCCCAGCCCCGCGCC
GCGCAGCAAACCATGCCCAAGTACGCTCATTGCTGCGTGGGTGCGTAAAATGCGGGTCAGTTGGCTGGAA
AGCAAATGCGACACACCTTTTGCCAATAATTTGTCTTTCATCAGCAGCGGCAGCAGCTCTTCCAGCTCAT
TCACCCTGGCATCGACCGCGTGCAGAAACTCCTGCTTATGTTCCTCGTCCATTTTCTTCCAGGTATTACG
CAGAAATTGTTCCAGTAACTGTTGCTCAATTTCAAACGTAGACATCTCTTTGTCGGCTTTCAGCTTCAAT
CGCTTTGAAACATCGAGCAAAATGGCCCGATACAATTTACCGTGTCCGCGCAGTTTGTTGGCGATACTAT
CGCCACCAAAATGCTGTAATTCTCCGGCAATCAGCTGCCAGTTGCGGCGATGTTGCTCGGGATGCCCTTC
CATCGATTTAAACAGTTCGTTGCGCATCAGTACGCTGGAGAGGCGAGTTTTGCCTTTTTCATTATGGGTG
AGCAATCGGGCGAAATTTGCCAACTGTTCCTCACTACAATGCTGAAGAAAATCCAGATCTGAATCATTCA
GGTAATTAACATTCATTTTTTGTGGCTTCTATATTCTGGCGTTAGTCGTCGCCGATAATTTTCAGCGTGG
CCATATCCGATGAGTTCACCGTATGACCCGAAAAGGTGATTTTTGAGACGCAGCGTTTATTGTCGTTATC
GCTGTTAATGTTGATCCAGTCAGTGGTTTGCCCTTCTTTTATTTCTGAAGGAATATTCAGGCTCTGACTG
GCGCTACGGGCGGCTTTGAAATAAACCGATGCACCGCTTAACTGTAAATCGCCATGGTCGGCAGAGAGTT
GTATGCGTTTCACAATGCGACAAACAGGAAGTTTCAGCGCCAGATCGTTGGTTTCGTTACGCGGCATTGC
AATGGCGCCGAGGAGTTTATGGTCGTTTGCCTGCGCCGTGCAGCACAGCATCAGGCTAATCGCCAGGCTG
GCGGAAATCGTAAAAACGGATTTCATAAGGATTCTCTTAGTGGGAAGAGGTAGGGGGATGAATACCCACT
AGTTTACTGCTGATAAAGAGAAGATTCAGGCACGTAATCTTTTCTTTTTATTACAATTTTTTGATGAATG
CCTTGGCTGCGATTCATTCTTTATATGAATAAAATTGCTGTCAATTTTACGTCTTGTCCTGCCATATCGC
GAAATTTCTGCGCAAAAGCACAAAAAATTTTTGCATCTCCCCCTTGATGACGTGGTTTACGACCCCATTT
AGTAGTCAACCGCAGTGAGTGAGTCTGCAAAAAAATGAAATTGGGCAGTTGAAACCAGACGTTTCGCCCC
TATTACAGACTCACAACCACATGATGACCGAATATATAGTGGAGACGTTTAGATGGGTAAAATAATTGGT
ATCGACCTGGGTACTACCAACTCTTGTGTAGCGATTATGGATGGCACCACTCCTCGCGTGCTGGAGAACG
CCGAAGGCGATCGCACCACGCCTTCTATCATTGCCTATACCCAGGATGGTGAAACTCTAGTTGGTCAGCC
GGCTAAACGTCAGGCAGTGACGAACCCGCAAAACACTCTGTTTGCGATTAAACGCCTGATTGGTCGCCGC
TTCCAGGACGAAGAAGTACAGCGTGATGTTTCCATCATGCCGTTCAAAATTATTGCTGCTGATAACGGCG
ACGCATGGGTCGAAGTTAAAGGCCAGAAAATGGCACCGCCGCAGATTTCTGCTGAAGTGCTGAAAAAAAT
GAAGAAAACCGCTGAAGATTACCTGGGTGAACCGGTAACTGAAGCTGTTATCACCGTACCGGCATACTTT
AACGATGCTCAGCGTCAGGCAACCAAAGACGCAGGCCGTATCGCTGGTCTGGAAGTAAAACGTATCATCA
ACGAACCGACCGCAGCTGCGCTGGCTTACGGTCTGGACAAAGGCACTGGCAACCGTACTATCGCGGTTTA
TGACCTGGGTGGTGGTACTTTCGATATTTCTATTATCGAAATCGACGAAGTTGACGGCGAAAAAACCTTC
GAAGTTCTGGCAACCAACGGTGATACCCACCTGGGGGGTGAAGACTTCGACAGCCGTCTGATCAACTATC
TGGTTGAAGAATTCAAGAAAGATCAGGGCATTGACCTGCGCAACGATCCGCTGGCAATGCAGCGCCTGAA
AGAAGCGGCAGAAAAAGCGAAAATCGAACTGTCTTCCGCTCAGCAGACCGACGTTAACCTGCCATACATC
ACTGCAGACGCGACCGGTCCGAAACACATGAACATCAAAGTGACTCGTGCGAAACTGGAAAGCCTGGTTG
AAGATCTGGTAAACCGTTCCATTGAGCCGCTGAAAGTTGCACTGCAGGACGCTGGCCTGTCCGTATCTGA
TATCGACGACGTTATCCTCGTTGGTGGTCAGACTCGTATGCCAATGGTTCAGAAGAAAGTTGCTGAGTTC
TTTGGTAAAGAGCCGCGTAAAGACGTTAACCCGGACGAAGCTGTAGCAATCGGTGCTGCTGTTCAGGGTG
GTGTTCTGACTGGTGACGTAAAAGACGTACTGCTGCTGGACGTTACCCCGCTGTCTCTGGGTATCGAAAC
CATGGGCGGTGTGATGACGACGCTGATCGCGAAAAACACCACTATCCCGACCAAGCACAGCCAGGTGTTC
TCTACCGCTGAAGACAACCAGTCTGCGGTAACCATCCATGTGCTGCAGGGTGAACGTAAACGTGCGGCTG
ATAACAAATCTCTGGGTCAGTTCAACCTAGATGGTATCAACCCGGCACCGCGCGGCATGCCGCAGATCGA
AGTTACCTTCGATATCGATGCTGACGGTATCCTGCACGTTTCCGCGAAAGATAAAAACAGCGGTAAAGAG
CAGAAGATCACCATCAAGGCTTCTTCTGGTCTGAACGAAGATGAAATCCAGAAAATGGTACGCGACGCAG
AAGCTAACGCCGAAGCTGACCGTAAGTTTGAAGAGCTGGTACAGACTCGCAACCAGGGCGACCATCTGCT
GCACAGCACCCGTAAGCAGGTTGAAGAAGCAGGCGACAAACTGCCGGCTGACGACAAAACTGCTATCGAG
TCTGCGCTGACTGCACTGGAAACTGCTCTGAAAGGTGAAGACAAAGCCGCTATCGAAGCGAAAATGCAGG
AACTGGCACAGGTTTCCCAGAAACTGATGGAAATCGCCCAGCAGCAACATGCCCAGCAGCAGACTGCCGG
TGCTGATGCTTCTGCAAACAACGCGAAAGATGACGATGTTGTCGACGCTGAATTTGAAGAAGTCAAAGAC
AAAAAATAATCGCCCTATAAACGGGTAATTATACTGACACGGGCGAAGGGGAATTTCCTCTCCGCCCGTG
CATTCATCTAGGGGCAATTTAAAAAAGATGGCTAAGCAAGATTATTACGAGATTTTAGGCGTTTCCAAAA
CAGCGGAAGAGCGTGAAATCAGAAAGGCCTACAAACGCCTGGCCATGAAATACCACCCGGACCGTAACCA
GGGTGACAAAGAGGCCGAGGCGAAATTTAAAGAGATCAAGGAAGCTTATGAAGTTCTGACCGACTCGCAA
A"""

In [5]:
long_str = long_str.replace("\n", "")

In [6]:
len(long_str)

14351